In [2]:
from typing import Annotated, Sequence, TypedDict, List, Dict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END, START
from langchain_groq import ChatGroq
from langgraph.prebuilt import ToolNode
import os
import json
import requests
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import MSO_ANCHOR
from io import BytesIO


In [3]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

llm = ChatGroq(model="llama-3.1-8b-instant")

In [4]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    subtopic: List[str]
    slide_segments: List[Dict[str, str]]
    ppt_output_path: str

In [5]:
def load_json_to_agent_state(json_path: str) -> AgentState:
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    complete_slide_segments = []
    for slide in data.get('slide_segments', []):
        complete_slide = {
            "slide_no": slide.get("slide_no", 0),
            "subtopic": slide.get("subtopic", ""),
            "content_to_display": slide.get("content_to_display", ""),
            "narration_script": slide.get("narration_script", ""),
            "is_blank_slide": slide.get("is_blank_slide", False),
            "image_address": slide.get("image_address", ""),
            "video_address": slide.get("video_address", ""),
            "image_position": slide.get("image_position", ""),
            "content_position": slide.get("test_position", "")
        }
        complete_slide_segments.append(complete_slide)

    return AgentState(
        messages=[],
        subtopic=data.get('subtopics', []),
        slide_segments=complete_slide_segments,
        ppt_output_path=""
    )


In [6]:
load_dotenv()
PEXELS_API_KEY = os.getenv("PEXELS_API_KEY")

PEXELS_IMAGE_API_URL = 'https://api.pexels.com/v1/search'
PEXELS_VIDEO_API_URL = 'https://api.pexels.com/videos/search'

def get_relevant_image(query: str, per_page: int = 1) -> str:
    headers = {'Authorization': PEXELS_API_KEY}
    params = {'query': query, 'per_page': per_page}
    response = requests.get(PEXELS_IMAGE_API_URL, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['photos']:
            return data['photos'][0]['src']['large']
    return ""

def get_relevant_video(query: str, per_page: int = 1) -> str:
    headers = {'Authorization': PEXELS_API_KEY}
    params = {'query': query, 'per_page': per_page}
    response = requests.get(PEXELS_VIDEO_API_URL, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['videos']:
            return data['videos'][0]['video_files'][0]['link']
    return ""


In [7]:
def enrich_all_slides_with_media(agent_state: AgentState) -> AgentState:
    """
    Enriches all slide segments in the agent_state with images or videos
    from Pexels based on whether the slide is blank or has content.
    """
    for slide in agent_state.get('slide_segments', []):
        if slide.get('is_blank_slide', False):
            slide['video_address'] = get_relevant_video(slide.get('subtopic', ''))
            slide['image_address'] = ""
            slide['image_position'] = ""
            slide['content_position'] = "center"
        else:
            slide['image_address'] = get_relevant_image(slide.get('content_to_display', ''))
            slide['video_address'] = ""
            slide['image_position'] = "right"
            slide['content_position'] = "left"
    return agent_state


In [8]:
import os
import requests
from io import BytesIO
import time
import pyautogui
import win32com.client
from pptx import Presentation
from pptx.util import Inches
from pptx.enum.text import MSO_ANCHOR

def create_ppt_with_designer(agent_state: dict, output_path: str = None) -> dict:
    if not output_path:
        output_path = os.path.join(os.getcwd(), 'GeneratedPresentation.pptx')

    # Step 1: Create presentation
    prs = Presentation()
    for slide_data in agent_state.get('slide_segments', []):
        slide = prs.slides.add_slide(prs.slide_layouts[5])  # Title Only layout
        title_shape = slide.shapes.add_textbox(Inches(0.5), Inches(0.3), Inches(9), Inches(1))
        title_shape.text_frame.text = slide_data.get('subtopic', '')

        content_text = slide_data.get('content_to_display', '')
        image_url = slide_data.get('image_address', '')
        image_position = slide_data.get('image_position', 'right')

        image_width = Inches(3)
        text_width = Inches(5.5)
        height = Inches(4)

        if image_position == 'left':
            img_left = Inches(0.5)
            text_left = img_left + image_width + Inches(0.5)
        else:
            text_left = Inches(0.5)
            img_left = text_left + text_width + Inches(0.5)

        # Add content text
        text_box = slide.shapes.add_textbox(text_left, Inches(1.5), text_width, height)
        text_box.text_frame.text = content_text
        text_box.text_frame.word_wrap = True
        text_box.text_frame.vertical_anchor = MSO_ANCHOR.TOP

        # Add image
        if image_url:
            try:
                img_data = requests.get(image_url).content
                img_stream = BytesIO(img_data)
                slide.shapes.add_picture(img_stream, img_left, Inches(1.5), width=image_width)
            except Exception as e:
                print(f"Failed to add image for slide {slide_data.get('slide_no')}: {e}")

    prs.save(output_path)

    # Step 2: Open PowerPoint and presentation
    ppt = win32com.client.Dispatch("PowerPoint.Application")
    ppt.Visible = True
    presentation = ppt.Presentations.Open(output_path)

    time.sleep(3)  # Let PowerPoint load

    # Step 3: Trigger Designer and apply the first suggestion
    try:
        pyautogui.hotkey('alt', 'g')         # Open "Design" tab
        time.sleep(1)
        pyautogui.hotkey('alt', 'g', 'd')    # Open "Design Ideas"
        time.sleep(4)                        # Wait for suggestions to load

        pyautogui.moveTo(1700, 200)          # Adjust coordinates if needed
        pyautogui.click()
        time.sleep(1)
    except Exception as e:
        print("Could not simulate Designer selection:", e)

    # Step 4: Save and close
    presentation.Save()
    presentation.Close()
    ppt.Quit()

    agent_state['ppt_output_path'] = output_path
    return agent_state


In [9]:
import os
import requests
from io import BytesIO
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import MSO_ANCHOR
from pptx.dml.color import RGBColor

def create_ppt_with_alignment(agent_state: dict, output_path: str = None) -> dict:
    if not output_path:
        output_path = os.path.join(os.getcwd(), 'GeneratedPresentation.pptx')

    prs = Presentation()

    for slide_data in agent_state.get('slide_segments', []):
        subtopic = slide_data.get('subtopic', '')

        # Skip 'BLANK_PAGE' slides
        
        slide = prs.slides.add_slide(prs.slide_layouts[5])

        # Title formatting
        title_shape = slide.shapes.add_textbox(Inches(0.5), Inches(0.3), Inches(9), Inches(1))
        title_frame = title_shape.text_frame
        title_frame.text = subtopic
        p = title_frame.paragraphs[0]
        run = p.runs[0]
        run.font.bold = True
        run.font.size = Pt(40)
        run.font.color.rgb = RGBColor(0, 0, 0)

        # Content and image placement
        content_text = slide_data.get('content_to_display', '')
        image_url = slide_data.get('image_address', '')
        image_position = slide_data.get('image_position', 'right')

        image_width = Inches(3)
        text_width = Inches(5.5)
        height = Inches(4)

        if image_position == 'left':
            img_left = Inches(0.5)
            text_left = img_left + image_width + Inches(0.5)
        else:
            text_left = Inches(0.5)
            img_left = text_left + text_width + Inches(0.5)

        # Add content text
        text_box = slide.shapes.add_textbox(text_left, Inches(1.5), text_width, height)
        tf = text_box.text_frame
        tf.text = content_text
        tf.word_wrap = True
        tf.vertical_anchor = MSO_ANCHOR.TOP

        # Add image
        if image_url:
            try:
                img_data = requests.get(image_url).content
                img_stream = BytesIO(img_data)
                slide.shapes.add_picture(img_stream, img_left, Inches(1.5), width=image_width)
            except Exception as e:
                print(f"Failed to add image for slide {slide_data.get('slide_no')}: {e}")

    prs.save(output_path)
    agent_state['ppt_output_path'] = output_path
    return agent_state


In [10]:
def load_data_node(state: AgentState) -> AgentState:
    return load_json_to_agent_state('../assets/scripts/slide_segments.json')

def enrich_media_node(state: AgentState) -> AgentState:
    return enrich_all_slides_with_media(state)

def generate_ppt_node(state: AgentState) -> AgentState:
    return create_ppt_with_alignment(state, output_path='../assets/ppts/GeneratedPresentation.pptx')

In [11]:
graph = StateGraph(AgentState)

graph.add_node("LoadData", load_data_node)
graph.add_node("EnrichMedia", enrich_media_node)
graph.add_node("GeneratePPT", generate_ppt_node)

graph.add_edge(START, "LoadData")
graph.add_edge("LoadData", "EnrichMedia")
graph.add_edge("EnrichMedia", "GeneratePPT")
# graph.add_edge("GeneratePPT", "ApplyDesigner")
graph.add_edge("GeneratePPT", END)

compiled_graph = graph.compile()

In [12]:
final_state = compiled_graph.invoke({})
print("Generated PPT Path:", final_state['ppt_output_path'])

Generated PPT Path: ../assets/ppts/GeneratedPresentation.pptx
